# Energy Saving Blueprint - PoC Demo

This notebook demonstrates intent-driven energy optimization in 5G networks.

**Workflow:**
1. Load 24-hour KPI data from CSV (UE and Cell reports)
2. Accept operator intent (QoS threshold)
3. Generate decisions for each 15-minute interval (96 total)
4. Output filtered decision table (Sleep/Wake actions only)
5. Execute decisions on RSG simulator at correct timestamps

## 1. Setup

In [ ]:
# Install dependencies
!pip install -q pandas sqlalchemy python-dotenv langchain-nvidia-ai-endpoints llama-index-llms-nvidia llama-index-embeddings-nvidia

In [ ]:
import pandas as pd
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Load environment variables
load_dotenv()

NVIDIA_API_KEY = ""

print(NVIDIA_API_KEY)

In [ ]:
# Configuration
import os
from pathlib import Path

PROJECT_ROOT = Path(os.getcwd()).parent if Path(os.getcwd()).name == "notebooks" else Path(os.getcwd())
DATA_PATH = str(PROJECT_ROOT / "data")
CELL_REPORTS = f"{DATA_PATH}/CellReports.csv"
UE_REPORTS = f"{DATA_PATH}/UEReports.csv"

TABLE_NAME = "kpi_data"
LLM_MODEL = os.getenv("LLM_MODEL", "meta/llama-3.1-70b-instruct")

# RSG / Scenario configuration
RSG_HOST = os.getenv("RSG_HOST", "3.211.96.252")  # AWS RSG host
USER_NAME = os.getenv("RSG_USER_NAME", os.getenv("USER", "user1"))
SCENARIO_CONF = os.getenv("SCENARIO_CONF", str(PROJECT_ROOT / "rsg_config" / "config.conf"))
RSG_ADDRESS = os.getenv("RSG_ADDRESS", "").strip()  # optional full override (internal RSG)


In [ ]:
# =============================================================================
# Setup — Install VIAVI ADK from RSG host (wheel)
# =============================================================================
import os, sys, subprocess

RSG_WHEEL_ADDRESS = os.getenv("RSG_WHEEL_ADDRESS", "http://3.211.96.252:8000").rstrip("/")
ADK_URL = f"{RSG_WHEEL_ADDRESS}/adk"

# NOTE: Do NOT set RSG_ADDRESS to the base host URL (e.g., http://3.211.96.252:8000).
#       RSG_ADDRESS is reserved for the *container* API root like http://<host>:8000/c/<hash>/
#       Use RSG_WHEEL_ADDRESS only for ADK wheel installation.
# Set to True only when you want to force reinstall/update
FORCE_ADK_UPDATE = False

def pip_install(args):
    subprocess.check_call([sys.executable, "-m", "pip", "install", *args])

def ensure_viavi_adk():
    if not FORCE_ADK_UPDATE:
        try:
            from viavi.rsg import Scenario, Simulation  # noqa: F401
            print("✓ VIAVI ADK already installed (viavi.rsg import OK)")
            return
        except Exception:
            pass

    print("Installing/updating VIAVI ADK from:", ADK_URL)
    pip_install(["-U", "setuptools", "packaging"])
    pip_install(["-U", ADK_URL])

    from viavi.rsg import Scenario, Simulation  # noqa: F401
    print("✓ VIAVI ADK installed/updated successfully")

ensure_viavi_adk()


## 2. Data Ingestion

In [ ]:
# Load UE and Cell reports
df_ue = pd.read_csv(UE_REPORTS)
df_cell = pd.read_csv(CELL_REPORTS)

print(f"UEReports: {len(df_ue)} rows, {df_ue['Viavi.UE.Name'].nunique()} unique UEs")
print(f"CellReports: {len(df_cell)} rows, {df_cell['Viavi.Cell.Name'].nunique()} unique cells")

# Filter active UEs (throughput > 0.1 Mbps)
ACTIVE_THROUGHPUT_THRESHOLD = 0.1
active_ues = df_ue[df_ue['DRB.UEThpDl'] > ACTIVE_THROUGHPUT_THRESHOLD].copy()
print(f"\nActive UEs (throughput > {ACTIVE_THROUGHPUT_THRESHOLD} Mbps): {active_ues['Viavi.UE.Name'].nunique()} unique")

# Get cell-level PRB from CellReports
cell_prb = df_cell[['time (ms)', 'Viavi.Cell.Name', 'RRU.PrbTotDl', 'Viavi.isEnergySaving']].copy()
cell_prb = cell_prb.rename(columns={'RRU.PrbTotDl': 'Cell_PRB'})

# Get UE-level throughput from active UEs
ue_thp = active_ues[['time (ms)', 'Viavi.Cell.Name', 'Viavi.UE.Name', 'DRB.UEThpDl']].copy()
ue_thp = ue_thp.rename(columns={'DRB.UEThpDl': 'UE_Throughput'})

# Merge UE data with Cell data (join on time and cell)
merged_df = ue_thp.merge(cell_prb, on=['time (ms)', 'Viavi.Cell.Name'], how='left')

# Parse site and band from cell name (e.g., "S1/N12/C1" -> site="S1/C1", band="N12")
merged_df['site'] = merged_df['Viavi.Cell.Name'].str.split('/').str[0] + '/' + merged_df['Viavi.Cell.Name'].str.split('/').str[2]
merged_df['band'] = merged_df['Viavi.Cell.Name'].str.split('/').str[1]

# Rename columns for SQL table
df = merged_df.rename(columns={
    'time (ms)': 'time',
    'Viavi.Cell.Name': 'cell_full',
    'Viavi.UE.Name': 'ue_name',
    'Viavi.isEnergySaving': 'is_sleeping'
})

print(f"\n✓ Merged data: {len(df)} active UE data points")
print(f"✓ Sites: {df['site'].nunique()}")
print(f"✓ Time points: {df['time'].nunique()}")
print(f"\nSample data:")
df[['time', 'site', 'band', 'ue_name', 'Cell_PRB', 'UE_Throughput']].head(10)

In [ ]:
# Store in SQLite
engine = create_engine('sqlite:///:memory:')
df.to_sql(TABLE_NAME, engine, if_exists='replace', index=False)

print(f"✓ Data stored in database table: {TABLE_NAME}")

## 3. Intent Input

In [ ]:
import re

# Get QoS range from data
min_qos = df['UE_Throughput'].min()
max_qos = df['UE_Throughput'].max()

print("="*60)
print("OPERATOR INTENT")
print("="*60)
print(f"\nNetwork QoS range: [{min_qos:.1f}, {max_qos:.1f}] Mbps")

# Get intent from user
operator_intent = input("\nEnter intent (or press Enter for default): ").strip()

# TODO:
# Add rejection for no itnent input
if not operator_intent:
    raise Exception("No intent was given.")

else:
    # Extract QoS threshold from intent
    qos_match = re.search(r'(\d+)\s*Mbps', operator_intent, re.IGNORECASE)
    if qos_match:
        QOS_THRESHOLD = float(qos_match.group(1))
    else:
        raise Exception("No threshold found.")

if QOS_THRESHOLD < min_qos or QOS_THRESHOLD > max_qos:
    raise Exception("Intent expectation is out of network range capabilities.")
print(f"\n✓ Intent: '{operator_intent}'")
print(f"✓ QoS Threshold: {QOS_THRESHOLD} Mbps")
print("="*60)

In [ ]:
# =============================================================================
# Virtual Time Zone (VTZ) — Map timestamps to traffic profiles
# =============================================================================
# Each 15-min interval in the dataset has a real UTC timestamp (e.g. 2023-01-01 01:00 UTC).
# VTZ maps any timestamp to the corresponding traffic profile so we know which
# RSG simulation configuration (UE count) to use.
#
# Traffic variation model:
#   - Throughput per UE is STATIC (constant across all periods)
#   - Number of active UEs varies per period via a multiplier (ue_ratio)
#   - ue_count = DEFAULT_UE_COUNT * ue_ratio
# =============================================================================
from datetime import datetime, timezone

# ---- Scenario defaults ----
DEFAULT_UE_COUNT = 200          # Base UE count from default scenario (active UEs in baseline)
DEFAULT_THROUGHPUT_MBPS = 8.0   # Static throughput per UE (constant for all periods)


# ---- Traffic Profiles (UE ratio model) ---- Early
traffic_profiles = [
    {"name": "Night",       "start_hour": 0,  "duration_hours": 8,  "ue_ratio": 0.1, "description": "Low traffic"},
    {"name": "Early",     "start_hour": 8,  "duration_hours": 1,  "ue_ratio": 1.5, "description": "Medium traffic"},
    {"name": "Morning",     "start_hour": 9,  "duration_hours": 4,  "ue_ratio": 1, "description": "Normal traffic"},
    {"name": "Afternoon",   "start_hour": 13, "duration_hours": 5,  "ue_ratio": 0.5, "description": "Normal traffic"},
    {"name": "Evening_1",     "start_hour": 18, "duration_hours": 2,  "ue_ratio": 2, "description": "Peak traffic"},
    {"name": "Evening_2",     "start_hour": 20, "duration_hours": 2,  "ue_ratio": 1, "description": "Normal traffic"},
    {"name": "Late Night",  "start_hour": 22, "duration_hours": 2,  "ue_ratio": 0.1, "description": "Low traffic"},

]

# Compute ue_count for each profile
for p in traffic_profiles:
    p["ue_count"] = int(DEFAULT_UE_COUNT * p["ue_ratio"])

# ---- Dict lookup by profile name: traffic_profiles_by_name["Night"] ----
traffic_profiles_by_name = {p["name"]: p for p in traffic_profiles}

# ---- VTZ mapping function ----
def vtz_get_profile(dt):
    """Map a datetime object to its traffic profile."""
    hour = dt.hour
    for p in traffic_profiles:
        if p["start_hour"] <= hour < p["start_hour"] + p["duration_hours"]:
            return {**p, "resolved_hour": hour, "resolved_datetime": dt}
    return None

# ---- Demo ----
print("="*70)
print("VIRTUAL TIME ZONE (VTZ) — Traffic Profile Mapping")
print("="*70)
print(f"\nScenario defaults:")
print(f"  Base UE count:          {DEFAULT_UE_COUNT}")
print(f"  Static throughput/UE:   {DEFAULT_THROUGHPUT_MBPS} Mbps")

print(f"\n{'Profile':<12} {'Hours':<14} {'UE Ratio':<10} {'UE Count':<10} {'Description'}")
print("-"*60)
for p in traffic_profiles:
    end = p['start_hour'] + p['duration_hours']
    print(f"{p['name']:<12} {p['start_hour']:02d}:00-{end:02d}:00   {p['ue_ratio']:<10.2f} {p['ue_count']:<10} {p['description']}")

# Example mappings
print(f"\nExample VTZ lookups:")
examples = [
    datetime(2023, 1, 1, 1, 0, tzinfo=timezone.utc),
    datetime(2023, 1, 1, 8, 30, tzinfo=timezone.utc),
    datetime(2023, 1, 1, 14, 0, tzinfo=timezone.utc),
    datetime(2023, 1, 1, 19, 0, tzinfo=timezone.utc),
]
for dt in examples:
    profile = vtz_get_profile(dt)
    print(f"  {dt.strftime('%Y-%m-%d %H:%M UTC')}  →  {profile['name']} (ratio={profile['ue_ratio']}, UEs={profile['ue_count']})")

print("="*70)

## 4. Closed-Loop Engine

In [ ]:
# Initialize NVIDIA LLM
llm = ChatNVIDIA(
    model=LLM_MODEL,
    api_key=NVIDIA_API_KEY,
    temperature=0.1,
    max_completion_tokens=1024
)

In [ ]:
# =============================================================================
# Closed-Loop Helper Functions
# =============================================================================
from dataclasses import dataclass, field
from datetime import datetime, timedelta, timezone
from typing import Dict, List, Optional
import time as time_module

@dataclass
class LoopState:
    """Carries state across closed-loop iterations."""
    cell_sleep_state: Dict[str, bool] = field(default_factory=dict)   # cell_name -> True if sleeping
    prev_kpis: Optional[Dict] = None                                   # KPIs from Sim #2 of previous iteration
    iteration_log: List[Dict] = field(default_factory=list)            # accumulated log


def datetime_to_baseline_timestamp(Tv, all_timestamps):
    """Map a Tv datetime to the nearest baseline dataset timestamp (ms)."""
    index = Tv.hour * 4 + Tv.minute // 15
    index = min(index, len(all_timestamps) - 1)
    return all_timestamps[index]


def get_baseline_kpis_for_interval(engine, timestamp_ms, df_ue, df_cell):
    """Extract per-site KPIs from baseline dataset for a specific 15-min interval."""
    # Filter baseline data to this timestamp
    ue_slice = df_ue[df_ue['time (ms)'] == timestamp_ms]
    cell_slice = df_cell[df_cell['time (ms)'] == timestamp_ms]

    if len(ue_slice) == 0 and len(cell_slice) == 0:
        return None

    # Overall metrics
    avg_thp = ue_slice['DRB.UEThpDl'].mean() if len(ue_slice) > 0 else 0.0

    # Per-site breakdown
    per_site = {}
    if len(cell_slice) > 0:
        for _, row in cell_slice.iterrows():
            cell_name = row['Viavi.Cell.Name']
            parts = cell_name.split('/')
            site = f"{parts[0]}/{parts[2]}"
            band = parts[1]  # N1 or N12

            if site not in per_site:
                per_site[site] = {'n1_prb': 0.0, 'n12_prb': 0.0, 'avg_qos': 0.0, 'n1_sleeping': False}

            if band == 'N1':
                per_site[site]['n1_prb'] = row.get('RRU.PrbTotDl', 0.0)
                per_site[site]['n1_sleeping'] = bool(row.get('Viavi.isEnergySaving', 0))
            elif band == 'N12':
                per_site[site]['n12_prb'] = row.get('RRU.PrbTotDl', 0.0)

    # Add avg QoS per site from UE data
    if len(ue_slice) > 0:
        for _, row in ue_slice.iterrows():
            cell_name = row['Viavi.Cell.Name']
            parts = cell_name.split('/')
            site = f"{parts[0]}/{parts[2]}"
            if site in per_site:
                # accumulate - will average later
                if 'qos_values' not in per_site[site]:
                    per_site[site]['qos_values'] = []
                per_site[site]['qos_values'].append(row['DRB.UEThpDl'])

    # Average QoS
    for site in per_site:
        if 'qos_values' in per_site[site]:
            per_site[site]['avg_qos'] = sum(per_site[site]['qos_values']) / len(per_site[site]['qos_values'])
            del per_site[site]['qos_values']

    sleeping_count = sum(1 for s in per_site.values() if s.get('n1_sleeping', False))
    total_count = len(per_site) * 2

    return {
        'avg_throughput_mbps': avg_thp,
        'sleeping_cells': sleeping_count,
        'total_cells': total_count,
        'per_site': per_site,
    }


def compute_kpis(sim_ue, sim_cell):
    """Transform raw RSG query results into structured KPI dict."""
    avg_thp = sim_ue['DRB.UEThpDl'].mean() if len(sim_ue) > 0 else 0.0

    per_site = {}
    if len(sim_cell) > 0:
        for _, row in sim_cell.iterrows():
            cell_name = row['Viavi.Cell.Name']
            parts = cell_name.split('/')
            site = f"{parts[0]}/{parts[2]}"
            band = parts[1]

            if site not in per_site:
                per_site[site] = {'n1_prb': 0.0, 'n12_prb': 0.0, 'avg_qos': 0.0, 'n1_sleeping': False}

            if band == 'N1':
                per_site[site]['n1_prb'] = row.get('RRU.PrbTotDl', 0.0)
                per_site[site]['n1_sleeping'] = bool(row.get('Viavi.isEnergySaving', 0))
            elif band == 'N12':
                per_site[site]['n12_prb'] = row.get('RRU.PrbTotDl', 0.0)

    # UE throughput per site
    if len(sim_ue) > 0:
        for _, row in sim_ue.iterrows():
            cell_name = row['Viavi.Cell.Name']
            parts = cell_name.split('/')
            site = f"{parts[0]}/{parts[2]}"
            if site in per_site:
                if 'qos_values' not in per_site[site]:
                    per_site[site]['qos_values'] = []
                per_site[site]['qos_values'].append(row['DRB.UEThpDl'])

    for site in per_site:
        if 'qos_values' in per_site[site]:
            per_site[site]['avg_qos'] = sum(per_site[site]['qos_values']) / len(per_site[site]['qos_values'])
            del per_site[site]['qos_values']

    sleeping_count = sum(1 for s in per_site.values() if s.get('n1_sleeping', False))

    return {
        'avg_throughput_mbps': avg_thp,
        'sleeping_cells': sleeping_count,
        'total_cells': len(per_site) * 2,
        'per_site': per_site,
    }


def format_kpis_for_prompt(kpis):
    """Format KPI dict into a readable table string for LLM prompts."""
    if kpis is None:
        return "No previous KPI data available."

    lines = []
    lines.append(f"Overall: avg_throughput={kpis['avg_throughput_mbps']:.2f} Mbps, "
                 f"sleeping_cells={kpis['sleeping_cells']}/{kpis['total_cells']}")

    if kpis.get('per_site'):
        lines.append(f"{'Site':<8} | {'N1 PRB':>7} | {'N12 PRB':>8} | {'Avg QoS':>9} | {'N1 State':>10}")
        lines.append("-" * 55)
        for site in sorted(kpis['per_site'].keys()):
            s = kpis['per_site'][site]
            state = "Sleeping" if s.get('n1_sleeping', False) else "Awake"
            lines.append(f"{site:<8} | {s['n1_prb']:>6.1f}% | {s['n12_prb']:>7.1f}% | "
                         f"{s['avg_qos']:>7.2f}   | {state:>10}")

    return "\n".join(lines)


def llm_generate_recommendations(llm, Tv, profile, loop_state, operator_intent,
                                  qos_threshold, all_sites, engine, df_ue, df_cell,
                                  all_timestamps):
    """LLM #1: Generate Sleep/Wake recommendations via SQL generation for current Tv interval."""
    import pandas as pd
    from sqlalchemy import text

    # Get baseline KPIs for this interval
    ts_ms = datetime_to_baseline_timestamp(Tv, all_timestamps)
    baseline_kpis = get_baseline_kpis_for_interval(engine, ts_ms, df_ue, df_cell)
    baseline_text = format_kpis_for_prompt(baseline_kpis) if baseline_kpis else "No baseline data for this interval."

    # Format previous iteration results
    prev_text = format_kpis_for_prompt(loop_state.prev_kpis) if loop_state.prev_kpis else "No previous iteration data (first iteration)."

    # ── Step 1: Write current cell_sleep_state into SQLite as cell_states table ──
    cell_states_rows = []
    for cell_name, is_sleeping in loop_state.cell_sleep_state.items():
        parts = cell_name.split("/")  # e.g. "S1/N1/C1" -> ["S1", "N1", "C1"]
        site = f"{parts[0]}/{parts[2]}" if len(parts) == 3 else cell_name
        band = parts[1] if len(parts) == 3 else ""
        cell_states_rows.append({
            'site': site,
            'band': band,
            'cell_full': cell_name,
            'current_is_sleeping': int(is_sleeping)
        })
    if cell_states_rows:
        df_states = pd.DataFrame(cell_states_rows)
    else:
        # Initial state: all awake
        df_states = pd.DataFrame([
            {'site': s, 'band': 'N1', 'cell_full': s.split('/')[0] + '/N1/' + s.split('/')[1], 'current_is_sleeping': 0}
            for s in all_sites
        ])
    df_states.to_sql('cell_states', engine, if_exists='replace', index=False)

    # ── Step 2: Build SQL generation prompt ──
    sleep_states_text = ""
    if loop_state.cell_sleep_state:
        sleeping = [c for c, v in loop_state.cell_sleep_state.items() if v]
        awake = [c for c, v in loop_state.cell_sleep_state.items() if not v]
        sleep_states_text = f"Currently sleeping: {sleeping if sleeping else 'None'}\nCurrently awake: {awake if awake else 'All cells awake'}"
    else:
        sleep_states_text = "All cells awake (initial state)."

    prompt_text = f"""**Operator Intent:** "{operator_intent}" (QoS threshold: {qos_threshold} Mbps)

**Current Time:** {Tv.strftime('%Y-%m-%d %H:%M UTC')}
**Traffic Profile:** {profile['name']} (UE ratio: {profile['ue_ratio']}, description: {profile['description']})

**Current Cell States:**
{sleep_states_text}

**Previous Iteration RSG Simulation Results:**
{prev_text}

**Baseline Dataset KPIs for this interval:**
{baseline_text}

**Task:** Generate a SQL query to analyze network conditions and make energy saving decisions for the current timestamp.

**Database Schema:**
- Table: kpi_data
  Columns: time (milliseconds), site, band ('N1' or 'N12'), cell_full, ue_name, Cell_PRB (%), UE_Throughput (Mbps), is_sleeping (0/1)
- Table: cell_states
  Columns: site, band, cell_full, current_is_sleeping (0=awake, 1=sleeping)

**IMPORTANT:** Use `band` column (not `cell`) to distinguish N1 vs N12.
**IMPORTANT:** Use `WHERE kd.time = {ts_ms}` to scope to the current 15-minute interval only.
**IMPORTANT:** JOIN with cell_states to get the current sleep state (do NOT use is_sleeping from kpi_data).

**Decision Rules (QoS threshold = {qos_threshold} Mbps):**
  SLEEP_CONDITION_1: (n1_prb = 0) AND (n12_prb = 0)
  SLEEP_CONDITION_2: (n12_prb < 35) AND (n1_prb < 12) AND (avg_qos >= {qos_threshold})

  For each site:
    IF current_is_sleeping = 1 AND n12_prb > 60 -> 'Wake'
    ELIF current_is_sleeping = 0 AND (SLEEP_CONDITION_1 OR SLEEP_CONDITION_2) -> 'Sleep'
    ELSE -> 'No Action'

**Required SQL Structure (follow this CTE pattern):**
```sql
WITH site_metrics AS (
  SELECT
    kd.site,
    COALESCE(MAX(CASE WHEN kd.band = 'N1' THEN kd.Cell_PRB END), 0) as n1_prb,
    COALESCE(MAX(CASE WHEN kd.band = 'N12' THEN kd.Cell_PRB END), 0) as n12_prb,
    COALESCE(AVG(kd.UE_Throughput), 0) as avg_qos,
    COALESCE(MAX(CASE WHEN cs.band = 'N1' THEN cs.current_is_sleeping END), 0) as current_is_sleeping
  FROM kpi_data kd
  LEFT JOIN cell_states cs ON kd.site = cs.site AND kd.band = cs.band
  WHERE kd.time = {ts_ms}
  GROUP BY kd.site
)
SELECT
  site,
  CASE
    WHEN current_is_sleeping = 1 AND n12_prb > 60 THEN 'Wake'
    WHEN current_is_sleeping = 0 AND ((n1_prb = 0 AND n12_prb = 0) OR (n12_prb < 35 AND n1_prb < 12 AND avg_qos >= {qos_threshold})) THEN 'Sleep'
    ELSE 'No Action'
  END as action,
  CASE
    WHEN current_is_sleeping = 1 AND n12_prb > 60 THEN 'N12 PRB=' || ROUND(n12_prb,1) || '% exceeds 60%, waking N1'
    WHEN current_is_sleeping = 0 AND n1_prb = 0 AND n12_prb = 0 THEN 'No traffic (N1 PRB=0%, N12 PRB=0%)'
    WHEN current_is_sleeping = 0 AND (n12_prb < 35 AND n1_prb < 12 AND avg_qos >= {qos_threshold}) THEN 'Low util (N1=' || ROUND(n1_prb,1) || '%, N12=' || ROUND(n12_prb,1) || '%, QoS=' || ROUND(avg_qos,2) || ' >= {qos_threshold})'
    ELSE 'No action needed'
  END as reason
FROM site_metrics
ORDER BY site;
```

**Output columns:** site, action, reason

Generate SQL following this exact structure. Return ONLY the SQL code."""

    prompt_template = ChatPromptTemplate.from_messages([
        ("system", "You are a SQL expert for SQLite databases. Generate clean, executable SQL queries. Return only the SQL code."),
        ("user", prompt_text)
    ])

    chain = prompt_template | llm | StrOutputParser()
    response = chain.invoke({})

    # ── Step 3: Extract SQL from response ──
    if "```sql" in response:
        generated_sql = response.split("```sql")[1].split("```")[0].strip()
    elif "```" in response:
        generated_sql = response.split("```")[1].split("```")[0].strip()
    else:
        generated_sql = response.strip()


    # Safety net: fix common LLM mistake of using "cell" instead of "band"
    import re
    generated_sql = re.sub(r'\bWHEN\s+cell\s*=', 'WHEN band =', generated_sql, flags=re.IGNORECASE)
    generated_sql = re.sub(r'\bkd\.cell\b', 'kd.band', generated_sql, flags=re.IGNORECASE)
    generated_sql = re.sub(r'\bcs\.cell\b', 'cs.band', generated_sql, flags=re.IGNORECASE)
    print(f"   Generated SQL: {generated_sql[:200]}...")

    # ── Step 4: Execute SQL against SQLite ──
    with engine.connect() as conn:
        result_df = pd.read_sql(text(generated_sql), conn)

    # Post-process: add target_cell column (S1/C1 → S1/N1/C1)
    if len(result_df) > 0:
        result_df['target_cell'] = result_df['site'].apply(
            lambda s: s.split('/')[0] + '/N1/' + s.split('/')[1]
        )
        # Filter to only Sleep/Wake rows (exclude No Action)
        result_df = result_df[result_df['action'].isin(['Sleep', 'Wake'])].reset_index(drop=True)
        result_df = result_df[['site', 'target_cell', 'action', 'reason']].drop_duplicates(
            subset=['target_cell', 'action']
        ).reset_index(drop=True)

    return result_df if len(result_df) > 0 else pd.DataFrame(columns=['site', 'target_cell', 'action', 'reason'])


def llm_validate_recommendations(llm, recommendations_df, sim1_kpis, Tv, profile,
                                  loop_state, operator_intent, qos_threshold):
    """LLM #2: Validate Sleep recommendations using Sim #1 KPIs (QoS check only).
    Returns approved-only DataFrame."""
    import pandas as pd

    if len(recommendations_df) == 0:
        return pd.DataFrame(columns=['site', 'target_cell', 'action', 'reason'])

    # Format recommendations + sim1 KPIs
    recs_text = ""
    for idx, row in recommendations_df.iterrows():
        site = row['site']
        site_kpis = sim1_kpis.get('per_site', {}).get(site, {})
        recs_text += (f"{idx+1} | {row['action']} | {row['target_cell']} | "
                      f"QoS:{site_kpis.get('avg_qos', 0):.2f}Mbps | {row['reason']}\n")

    sim_summary = format_kpis_for_prompt(sim1_kpis)

    prompt_text = f"""**Operator Intent:** "{operator_intent}" (QoS threshold: {qos_threshold} Mbps)
**Current Time:** {Tv.strftime('%Y-%m-%d %H:%M UTC')}
**Traffic Profile:** {profile['name']}

**RSG Simulation #1 Results (testing Sleep recommendations):**
{sim_summary}

**Recommendations to Validate (# | ACTION | CELL | QoS | REASON):**
{recs_text}

**Task:** For each Sleep recommendation, decide YES (approve) or NO (reject).

NOTE: N1 cells are already sleeping in Sim #1 results, so N1 PRB will be 0 by
definition. Focus on N12 PRB and QoS to judge impact.

**Validation Rules (evaluate in order, stop at first match):**
APPROVE_RULE_1: N12_PRB == 0 → YES (no traffic at site, Sleep has zero impact)
APPROVE_RULE_2: AVG_QOS >= {qos_threshold} Mbps → YES (QoS maintained despite Sleep)
REJECT_RULE:    AVG_QOS < {qos_threshold} Mbps AND N12_PRB > 0 → NO (Sleep degraded QoS)

**Examples:**
1 | Sleep | S1/N1/C1 | N12_PRB=0.0%, QoS=N/A   → 1 | YES | No traffic at site (APPROVE_RULE_1)
2 | Sleep | S2/N1/C1 | N12_PRB=25%, QoS=5.20    → 2 | YES | QoS 5.20 >= {qos_threshold} (APPROVE_RULE_2)
3 | Sleep | S3/N1/C1 | N12_PRB=45%, QoS=1.50    → 3 | NO  | QoS 1.50 < {qos_threshold}, site has traffic (REJECT_RULE)
4 | Sleep | S4/N1/C1 | N12_PRB=12%, QoS={qos_threshold - 0.1:.1f}    → 4 | NO  | QoS {qos_threshold - 0.1:.1f} < {qos_threshold}, site has traffic
(REJECT_RULE)

Now validate the actual recommendations:

Reply ONLY in format: NUM | YES/NO | RULE_NAME | reason"""

    prompt_template = ChatPromptTemplate.from_messages([
        ("system", "You are a 5G network validation expert. Approve or reject Sleep recommendations based on QoS impact from RSG simulation results. Reply in format: NUM | YES/NO | reason"),
        ("user", prompt_text)
    ])

    chain = prompt_template | llm | StrOutputParser()
    response = chain.invoke({})

    # Parse response
    approved_indices = set()
    for line in response.strip().split('\n'):
        line = line.strip()
        if '|' in line:
            parts = [p.strip() for p in line.split('|')]
            if len(parts) >= 2:
                try:
                    num = int(parts[0].replace('.', '').strip())
                    decision = parts[1].strip().upper()
                    if 'YES' in decision:
                        approved_indices.add(num - 1)  # convert to 0-based
                except (ValueError, IndexError):
                    pass

    # Filter to approved only (guard against out-of-bounds indices from LLM)
    valid_indices = sorted(i for i in approved_indices if 0 <= i < len(recommendations_df))
    if valid_indices:
        approved_df = recommendations_df.iloc[valid_indices].copy()
    else:
        approved_df = pd.DataFrame(columns=['site', 'target_cell', 'action', 'reason'])

    return approved_df
  

_original_ue_distribution = None

# =============================================================================
# IMPROVED run_sim — Persistent simulation with warm-up & aggregation
# =============================================================================

_persistent_sim = None          # Keep simulation alive across iterations
_original_ue_distribution = None
SIM_DURATION = 10               # RSG default: 10s simulation, last second is steady-state


def run_sim(scenario, profile, actions_to_apply, loop_state):
  """Run RSG simulation with default 10s duration.

  RSG stabilizes within 10 seconds. Query start=9, stop=10 returns
  the last second of steady-state data (42 rows for CellReports,
  one per cell).
  """
  global _persistent_sim, _original_ue_distribution

  # 1. Scale UE count
  if _original_ue_distribution is None:
      _original_ue_distribution = []
      for ue_group in scenario.config["UE_Configuration"]["UE_Groups"]:
          group_dists = []
          for dist in ue_group.get("distribution", []):
              group_dists.append(dist["ues"])
          _original_ue_distribution.append(group_dists)

  for g_idx, ue_group in enumerate(scenario.config["UE_Configuration"]["UE_Groups"]):
      for d_idx, dist in enumerate(ue_group.get("distribution", [])):
          original_ues = _original_ue_distribution[g_idx][d_idx]
          dist["ues"] = max(1, int(original_ues * profile['ue_ratio']))
      if "serviceConfig" in ue_group:
          for service in ue_group["serviceConfig"]:
              service["targetTput_Mbps"] = DEFAULT_THROUGHPUT_MBPS

  # 2. Configure batch mode
  scenario.config['System']['batch_mode'] = True
  scenario.config['System']['duration'] = SIM_DURATION

  # 3. Start simulation (fresh start needed because UE config changed)
  sim = scenario.simulation(force_start=True, adk_pace=True)
  sim.start()

  # 4. Re-apply accumulated sleep states
  for cell_name, is_sleeping in loop_state.cell_sleep_state.items():
      if is_sleeping:
          sim.command('turn_off', cell=cell_name, reason='Accumulated sleep state')
      else:
          sim.command('turn_on', cell=cell_name, reason='Accumulated awake state')

  # 5. Apply new actions
  if actions_to_apply is not None and len(actions_to_apply) > 0:
      for _, row in actions_to_apply.iterrows():
          target_cell = row['target_cell']
          action = row['action']
          if action == 'Sleep':
              sim.command('turn_off', cell=target_cell,
                         reason=row.get('reason', 'Energy optimization'))
          elif action == 'Wake':
              sim.command('turn_on', cell=target_cell,
                         reason=row.get('reason', 'Capacity needed'))

  # 6. Run for default duration
  sim.run_for(f"{SIM_DURATION}s")

  # 7. Query last second of steady-state data
  sim_ue = sim.query("UEReports", start=9, stop=10)
  sim_cell = sim.query("CellReports", start=9, stop=10)

  # 8. Stop simulation
  sim.finish()

  # 9. Compute KPIs from last-second snapshot
  # Deduplicate: RSG returns multiple samples per cell
  sim_cell = sim_cell.drop_duplicates(subset=['Viavi.Cell.Name'], keep='last').reset_index(drop=True)
  return compute_kpis(sim_ue, sim_cell), sim_cell

In [ ]:
def compute_energy_efficiency(summary_df):
    """Compute energy efficiency KPI based on % of sleeping cells over the simulation window."""
    if summary_df.empty:
        print("\n  No iteration data — cannot compute energy efficiency.")
        return

    from collections import defaultdict

    ratios = summary_df['sleeping_cells'] / summary_df['total_cells']
    avg_ratio = ratios.mean()
    num_intervals = len(summary_df)
    window_hours = num_intervals * 15 / 60
    total_cells = int(summary_df['total_cells'].iloc[0])

    print(f"\nENERGY EFFICIENCY KPI (Sleeping Cell Ratio)")
    print("─" * 60)
    print(f"  Window: {num_intervals} intervals ({window_hours:.1f} hours)")
    print(f"  Total cells: {total_cells} (21 N1 + 21 N12)")
    print(f"  Theoretical max: 50.0% (only N1 cells can sleep)")
    print(f"  Average sleeping cell ratio: {avg_ratio * 100:.1f}%")
    print(f"  → Estimated energy saving potential: ~{avg_ratio * 100:.1f}%")

    # Per-profile breakdown
    profile_ratios = defaultdict(list)
    for _, row in summary_df.iterrows():
        dt = datetime.strptime(row['time_utc'], '%Y-%m-%d %H:%M')
        profile = vtz_get_profile(dt.replace(tzinfo=timezone.utc))
        if profile:
            ratio = row['sleeping_cells'] / row['total_cells']
            profile_ratios[profile['name']].append(ratio)

    if profile_ratios:
        print(f"\n  Per-profile breakdown:")
        for p in traffic_profiles:
            name = p['name']
            if name in profile_ratios:
                avg = sum(profile_ratios[name]) / len(profile_ratios[name])
                count = len(profile_ratios[name])
                print(f"    {name:<12} ({count:>2} intervals): {avg * 100:5.1f}% cells sleeping")

In [ ]:
# =============================================================================
# Main Closed-Loop Function
# =============================================================================

def run_closed_loop(llm, scenario, engine, df_ue, df_cell, all_timestamps,
                    all_sites, operator_intent, qos_threshold,
                    start_datetime, num_iterations):
    """
    Run the closed-loop per-interval iteration engine.

    7-step loop per iteration:
      1. LLM #1: Generate Sleep/Wake recommendations
      2. Blueprint picks (implicit — LLM is profile-aware)
      3. RSG Sim #1: Test Sleep recommendations with current profile
         (Wake recs bypass — wake is non-destructive per Flow.md)
      4. LLM #2: Validate Sleep recs using Sim #1 KPIs → approved list
         Wake recs auto-approved and concatenated
      5. Advance Tv by 15 min
      6. RSG Sim #2: Apply approved actions with next profile
      7. Feed Sim #2 KPIs back to next iteration

    Returns:
        (loop_state, summary_df) — LoopState and DataFrame of iteration summaries
    """
    import pandas as pd

    loop_state = LoopState()
    Tv = start_datetime

    print("=" * 80)
    print("CLOSED-LOOP PER-INTERVAL ENGINE")
    print("=" * 80)
    print(f"Start time: {Tv.strftime('%Y-%m-%d %H:%M UTC')}")
    print(f"Iterations: {num_iterations}")
    print(f"Operator intent: {operator_intent}")
    print(f"QoS threshold: {qos_threshold} Mbps")
    print("=" * 80)

    engine_start = time_module.time()

    # Persistent log file
    log_dir = str(PROJECT_ROOT / 'output')
    os.makedirs(log_dir, exist_ok=True)
    log_path = f"{log_dir}/closed_loop_log.txt"
    log_file = open(log_path, 'w')
    log_file.write(f"CLOSED-LOOP SIMULATION LOG\n")
    log_file.write(f"Start: {start_datetime.strftime('%Y-%m-%d %H:%M UTC')}\n")
    log_file.write(f"Iterations: {num_iterations}\n")
    log_file.write(f"Intent: {operator_intent}\n")
    log_file.write(f"QoS threshold: {qos_threshold} Mbps\n")
    log_file.write("=" * 80 + "\n")
    log_file.flush()

    for iteration in range(num_iterations):
        iter_start = time_module.time()
        profile = vtz_get_profile(Tv)
        if profile is None:
            print(f"\n⚠ Iteration {iteration+1}: No profile for {Tv}, skipping")
            Tv = Tv + timedelta(minutes=15)
            continue

        print(f"\n{'─' * 80}")
        print(f"ITERATION {iteration+1}/{num_iterations} | {Tv.strftime('%H:%M UTC')} | Profile: {profile['name']} (ratio={profile['ue_ratio']})")
        print(f"{'─' * 80}")

        sleeping_count = sum(1 for v in loop_state.cell_sleep_state.values() if v)
        print(f"  Current state: {sleeping_count} cells sleeping")

        # Seed prev_kpis from baseline on first iteration
        if loop_state.prev_kpis is None:
            ts_ms = datetime_to_baseline_timestamp(Tv, all_timestamps)
            loop_state.prev_kpis = get_baseline_kpis_for_interval(engine, ts_ms, df_ue, df_cell)
            print(f"  Stored prev_kpis from baseline (ts={ts_ms})")

        # Fetch baseline KPIs for this interval (for logging)
        ts_ms = datetime_to_baseline_timestamp(Tv, all_timestamps)
        baseline_kpis = get_baseline_kpis_for_interval(engine, ts_ms, df_ue, df_cell)
        if iteration == 0:
            _bl_sites = len(baseline_kpis.get('per_site', {})) if baseline_kpis else 0
            _bl_cells = baseline_kpis.get('total_cells', 0) if baseline_kpis else 0
            print(f"  [BASELINE DEBUG] sites={_bl_sites}, total_cells={_bl_cells}, "
                  f"df_cell rows at ts={len(df_cell[df_cell['time (ms)']==ts_ms])}")

        # ── Step 1: LLM #1 — Generate recommendations ──
        print(f"  Step 1: LLM generating recommendations...")
        t0 = time_module.time()
        recommendations = llm_generate_recommendations(
            llm, Tv, profile, loop_state, operator_intent,
            qos_threshold, all_sites, engine, df_ue, df_cell, all_timestamps
        )
        t_llm1 = time_module.time() - t0

        # Split: Wake bypasses Sim #1 + LLM #2 (wake is non-destructive)
        sleep_recs = recommendations[recommendations['action'] == 'Sleep'].drop_duplicates(subset=['target_cell']).reset_index(drop=True)
        wake_recs = recommendations[recommendations['action'] == 'Wake'].drop_duplicates(subset=['target_cell']).reset_index(drop=True)
        n_sleep, n_wake = len(sleep_recs), len(wake_recs)

        parts = []
        if n_sleep > 0:
            parts.append(f"{n_sleep} Sleep")
        if n_wake > 0:
            parts.append(f"{n_wake} Wake (auto-approved)")
        count_str = ", ".join(parts) if parts else "0"
        print(f"    → {len(recommendations)} recommendations generated: {count_str} ({t_llm1:.1f}s)")
        if len(recommendations) > 0:
            for _, r in recommendations.iterrows():
                tag = " [auto-approved]" if r['action'] == 'Wake' else ""
                print(f"      {r['action']:5s} {r['target_cell']} — {r.get('reason', '')[:60]}{tag}")

        # ── Step 2: Blueprint picks (implicit in LLM prompt) ──

        # ── Step 3: RSG Sim #1 — Test Sleep recommendations only ──
        t_sim1 = 0.0
        if n_sleep > 0:
            print(f"  Step 3: RSG Sim #1 (testing {n_sleep} Sleep recommendations)...")
            t0 = time_module.time()
            sim1_kpis, sim1_cell = run_sim(scenario, profile, sleep_recs, loop_state)
            t_sim1 = time_module.time() - t0
            print(f"    → avg_thp={sim1_kpis['avg_throughput_mbps']:.2f} Mbps, "
                  f"sleeping={sim1_kpis['sleeping_cells']}/{sim1_kpis['total_cells']} ({t_sim1:.1f}s)")
            if len(sim1_cell) > 0:
                for _, sr in sleep_recs.iterrows():
                    cell_row = sim1_cell[sim1_cell['Viavi.Cell.Name'] == sr['target_cell']]

        else:
            sim1_kpis = loop_state.prev_kpis or {}
            print(f"  Step 3: Skipped (no Sleep recommendations)")

        t_llm2 = 0.0
        # ── Step 4: LLM #2 — Validate Sleep recommendations only ──
        if n_sleep > 0:
            print(f"  Step 4: LLM validating {n_sleep} Sleep recommendations...")
            t0 = time_module.time()
            approved_sleep = llm_validate_recommendations(
                llm, sleep_recs, sim1_kpis, Tv, profile,
                loop_state, operator_intent, qos_threshold
            )
            t_llm2 = time_module.time() - t0
        else:
            approved_sleep = pd.DataFrame(columns=['site', 'target_cell', 'action', 'reason'])

        # Wake always approved — concat with validated Sleep
        approved = pd.concat([approved_sleep, wake_recs], ignore_index=True)

        # Print Step 4 summary
        if n_sleep > 0 or n_wake > 0:
            parts = []
            if n_sleep > 0:
                parts.append(f"{len(approved_sleep)}/{n_sleep} Sleep approved")
            if n_wake > 0:
                parts.append(f"{n_wake} Wake (auto-approved)")
            summary = " + ".join(parts) + f" = {len(approved)} total"
            print(f"    → {summary} ({t_llm2:.1f}s)")
            if len(approved) > 0:
                for _, a in approved.iterrows():
                    tag = " [auto]" if a['action'] == 'Wake' else ""
                    print(f"      ✓ {a['action']:5s} {a['target_cell']}{tag}")
        else:
            print(f"  Step 4: Skipped (no recommendations)")

        # ── Step 5: Advance Tv ──
        Tv = Tv + timedelta(minutes=15)
        next_profile = vtz_get_profile(Tv)
        if next_profile is None:
            # Wrap around to start of day
            Tv = Tv.replace(hour=0, minute=0)
            next_profile = vtz_get_profile(Tv)
        print(f"  Step 5: Tv → {Tv.strftime('%H:%M UTC')} (profile: {next_profile['name']})")

        # ── Step 6: RSG Sim #2 — Apply approved actions ──
        # Update cell_sleep_state ONLY from approved actions
        if len(approved) > 0:
            for _, row in approved.iterrows():
                cell_name = row['target_cell']
                if row['action'] == 'Sleep':
                    loop_state.cell_sleep_state[cell_name] = True
                elif row['action'] == 'Wake':
                    loop_state.cell_sleep_state[cell_name] = False

        print(f"  Step 6: RSG Sim #2 (approved actions + next profile)...")
        t0 = time_module.time()
        sim2_kpis, sim2_cell = run_sim(scenario, next_profile, None, loop_state)
        t_sim2 = time_module.time() - t0
        # NOTE: actions already in cell_sleep_state, no new actions passed
        print(f"    → avg_thp={sim2_kpis['avg_throughput_mbps']:.2f} Mbps, "
              f"sleeping={sim2_kpis['sleeping_cells']}/{sim2_kpis['total_cells']} ({t_sim2:.1f}s)")
        if len(approved) > 0 and len(sim2_cell) > 0:
            for _, ar in approved.iterrows():
                cell_row = sim2_cell[sim2_cell['Viavi.Cell.Name'] == ar['target_cell']]

        # ── Step 7: Feed back ──
        loop_state.prev_kpis = sim2_kpis

        sleeping_after = sum(1 for v in loop_state.cell_sleep_state.values() if v)
        iter_elapsed = time_module.time() - iter_start
   
        loop_state.iteration_log.append({
            'iteration': iteration + 1,
            'time_utc': (Tv - timedelta(minutes=15)).strftime('%Y-%m-%d %H:%M'),
            'sleeping_cells': sleeping_after,
            'total_cells': len(all_sites) * 2,
            'baseline_avg_thp': baseline_kpis.get('avg_throughput_mbps', 0) if baseline_kpis else 0,
            'sim1_avg_thp': sim1_kpis.get('avg_throughput_mbps', 0),
            'sim2_avg_thp': sim2_kpis['avg_throughput_mbps'],
            'recommendations_generated': len(recommendations),
            'recommendations_approved': len(approved),
        })
        total_elapsed = time_module.time() - engine_start
        avg_per_iter = total_elapsed / (iteration + 1)
        remaining = avg_per_iter * (num_iterations - iteration - 1)
        print(f"  ✓ Iteration {iteration+1} complete — {sleeping_after} cells sleeping")
        print(f"    Timing: LLM1={t_llm1:.1f}s  Sim1={t_sim1:.1f}s  LLM2={t_llm2:.1f}s  Sim2={t_sim2:.1f}s  |  Iter={iter_elapsed:.1f}s")
        print(f"    Elapsed: {total_elapsed:.0f}s ({total_elapsed/60:.1f}min)  |  ETA: {remaining:.0f}s ({remaining/60:.1f}min)")

        # Write iteration to log file
        log_file.write(f"\nIteration {iteration+1}/{num_iterations} | {(Tv - timedelta(minutes=15)).strftime('%H:%M UTC')} | {profile['name']}\n")
        log_file.write(f"  sleeping_cells={sleeping_after}/{len(all_sites)*2}\n")
        log_file.write(f"  baseline: avg_thp={baseline_kpis.get('avg_throughput_mbps', 0) if baseline_kpis else 0:.2f} Mbps\n")
        log_file.write(f"  sim1: avg_thp={sim1_kpis.get('avg_throughput_mbps', 0):.2f} Mbps\n")
        log_file.write(f"  sim2: avg_thp={sim2_kpis['avg_throughput_mbps']:.2f} Mbps\n")
        log_file.write(f"  timing: LLM1={t_llm1:.1f}s Sim1={t_sim1:.1f}s LLM2={t_llm2:.1f}s Sim2={t_sim2:.1f}s | Total={iter_elapsed:.1f}s\n")
        log_file.flush()

    total_time = time_module.time() - engine_start
    summary_df = pd.DataFrame(loop_state.iteration_log)

    print("\n" + "=" * 80)
    print("CLOSED-LOOP ENGINE COMPLETE")
    print("=" * 80)
    print(f"Total iterations: {num_iterations}")
    print(f"Final sleeping cells: {sum(1 for v in loop_state.cell_sleep_state.values() if v)}")
    print(f"Total wall time: {total_time:.1f}s ({total_time/60:.1f}min)")
    print(f"Avg per iteration: {total_time/max(num_iterations,1):.1f}s")

    # Energy efficiency KPI
    compute_energy_efficiency(summary_df)

    print("=" * 80)

    # Write ITERATION SUMMARY and final table to log
    log_file.write("\n" + "=" * 80 + "\n")
    log_file.write("ITERATION SUMMARY\n")
    log_file.write("=" * 80 + "\n")
    log_file.write(summary_df.to_string(index=False) + "\n")
    log_file.write("=" * 80 + "\n")
    log_file.write(f"\nTotal iterations: {num_iterations}\n")
    log_file.write(f"Final sleeping cells: {sum(1 for v in loop_state.cell_sleep_state.values() if v)}\n")
    log_file.write(f"Total wall time: {total_time:.1f}s ({total_time/60:.1f}min)\n")
    log_file.close()
    print(f"\n✓ Log written to {log_path}")

    return loop_state, summary_df

## 5. Execute Closed-Loop Simulation

Run the closed-loop engine: each 15-minute interval goes through LLM recommendation → RSG simulation → LLM validation → RSG confirmation.

In [ ]:
# =============================================================================
# Execution — Closed-Loop Simulation (AWS RSG default)
# =============================================================================
import os, json, hashlib, requests, socket
from pathlib import Path
from viavi.rsg import Scenario, Simulation  # noqa: F401
from datetime import datetime, timezone

def _tcp_ok(host: str, port: int = 8000, timeout: float = 5.0) -> bool:
    try:
        socket.create_connection((host, port), timeout=timeout).close()
        return True
    except Exception as ex:
        print(f"❌ TCP connect failed to {host}:{port}: {ex}")
        return False

def _resolve_conf_arg(conf_name: str) -> str:
    """Prefer an absolute local path if the conf exists; otherwise pass through the name."""
    conf_name = (conf_name or "").strip()
    if not conf_name:
        return conf_name
    p = Path(conf_name)
    if p.exists():
        return str(p.resolve())
    # try relative to PROJECT_ROOT and cwd
    for base in [Path.cwd(), PROJECT_ROOT if 'PROJECT_ROOT' in globals() else Path.cwd()]:
        cand = base / conf_name
        if cand.exists():
            return str(cand.resolve())
    # fallback: if any *.conf exists in cwd, use most recent
    confs = sorted(Path.cwd().glob('*.conf'), key=lambda x: x.stat().st_mtime, reverse=True)
    if confs:
        print(f"⚠ Conf '{conf_name}' not found; using most recent conf in cwd: {confs[0].name}")
        return str(confs[0].resolve())
    return conf_name

def connect_rsg_and_load_scenario(rsg_host: str, user_name: str, scenario_conf: str, rsg_address_override: str = ""):
    """Connect to RSG and create a Scenario, printing actionable diagnostics on failure."""
    scenario_arg = _resolve_conf_arg(scenario_conf)

    # If caller provides a full RSG base URL override (e.g., internal RSG), use it directly.
    if rsg_address_override:
        print("Using RSG_ADDRESS override:", rsg_address_override)
        return Scenario(scenario_arg, rsg_address_override)

    # Network preflight
    if not _tcp_ok(rsg_host, 8000):
        raise RuntimeError(f"Cannot reach RSG host {rsg_host}:8000 (network/VPN/firewall).")

    # Best-effort status
    try:
        s = requests.get(f"http://{rsg_host}:8000/status", timeout=10)
        print("/status HTTP:", s.status_code)
        if s.ok:
            status = s.json()
            print(f"RSG containers: {status.get('available_containers')}/{status.get('total_containers')}")
            if status.get('message'):
                print("RSG message:", status['message'])
        else:
            print("/status body:", s.text[:500])
    except Exception as ex:
        print(f"⚠ Could not query /status: {ex}")

    # Allocate a stable session per (public_ip, user_name)
    try:
        ip = requests.get("https://api.ipify.org", timeout=5).text.strip()
    except Exception:
        ip = "unknownip"
    h = hashlib.sha256(f"{ip}-{user_name}".encode()).hexdigest()[:8]
    rsg_addr = f"http://{rsg_host}:8000/c/{h}/"
    print("RSG addr:", rsg_addr)
    print("Scenario arg:", scenario_arg)

    try:
        return Scenario(scenario_arg, rsg_addr)
    except Exception as e:
        resp = getattr(getattr(e, '__cause__', None), 'response', None)
        if resp is not None:
            try:
                payload = resp.json()
                detail = json.dumps(payload, indent=2)[:2000]
            except Exception:
                detail = resp.text[:2000]
            raise RuntimeError(f"Scenario creation failed (HTTP {resp.status_code}). Details:\n{detail}") from e
        raise

# Create Scenario
scenario = connect_rsg_and_load_scenario(
    rsg_host=RSG_HOST,
    user_name=USER_NAME,
    scenario_conf=SCENARIO_CONF,
    rsg_address_override=RSG_ADDRESS,
)
print("✓ Scenario connected")

# Pre-calculate avg QoS (needed by helper functions)
avg_qos = df.groupby(['time', 'site'])['UE_Throughput'].mean().reset_index()
avg_qos = avg_qos.rename(columns={'UE_Throughput': 'avg_qos'})

from sqlalchemy import text
with engine.connect() as conn:
    conn.execute(text("DROP TABLE IF EXISTS site_avg_qos"))
    conn.commit()
avg_qos.to_sql('site_avg_qos', engine, index=False, if_exists='replace')
print(f"✓ Pre-calculated avg QoS for {len(avg_qos)} (time, site) combinations")

# All timestamps and sites from dataset
all_timestamps = sorted(df['time'].unique())
all_sites = sorted(df['site'].unique())

# ms_to_datetime utility
def ms_to_datetime(ms):
    """Convert milliseconds timestamp to UTC datetime string."""
    return datetime.fromtimestamp(ms / 1000, tz=timezone.utc).strftime('%Y-%m-%d %H:%M UTC')

print(f"✓ {len(all_timestamps)} timestamps, {len(all_sites)} sites")

# ── User Configuration ──
START_DATETIME = datetime(2023, 1, 1, 0, 0, tzinfo=timezone.utc)
NUM_ITERATIONS = 96  # Full 24h (set to 4 for quick test = 1 hour)

print(f"\nStarting closed-loop: {START_DATETIME.strftime('%Y-%m-%d %H:%M UTC')}, {NUM_ITERATIONS} iterations")

# ── Execute ──
loop_state, summary_df = run_closed_loop(
    llm=llm,
    scenario=scenario,
    engine=engine,
    df_ue=df_ue,
    df_cell=df_cell,
    all_timestamps=all_timestamps,
    all_sites=all_sites,
    operator_intent=operator_intent,
    qos_threshold=QOS_THRESHOLD,
    start_datetime=START_DATETIME,
    num_iterations=NUM_ITERATIONS,
)

# Display summary
print("\n" + "=" * 100)
print("ITERATION SUMMARY")
print("=" * 100)
print(summary_df.to_string(index=False))
print("=" * 100)


In [ ]:
!pip install matplotlib

In [ ]:
# =============================================================================
# Closed-Loop Results Visualization
# =============================================================================
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.ticker as mticker
import numpy as np

# Traffic period colors
_PERIODS = [
    {"name": "Night",      "start": 0,  "end": 8,  "color": "#1a237e"},
    {"name": "Early",    "start": 8,  "end": 9,  "color": "#b71c1c"},
    {"name": "Morning",    "start": 9,  "end": 13, "color": "#e65100"},
    {"name": "Afternoon",  "start": 13, "end": 18, "color": "#1b5e20"},
    {"name": "Evening_1",    "start": 18, "end": 20, "color": "#4a148c"},
    {"name": "Evening_2",    "start": 20, "end": 22, "color": "#4a148c"},
    {"name": "Late Night", "start": 22, "end": 24, "color": "#1a237e"},
]

# Extract time axis from summary_df
time_hours = []
for t in summary_df['time_utc']:
    dt = pd.to_datetime(t)
    time_hours.append(dt.hour + dt.minute / 60)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 9), sharex=True)

# ── Panel 1: Sleeping cells over time ──
ax1.fill_between(time_hours, summary_df['sleeping_cells'], alpha=0.35, color='#1565C0')
ax1.plot(time_hours, summary_df['sleeping_cells'], color='#0D47A1', linewidth=2)

for p in _PERIODS:
    ax1.axvspan(p['start'], p['end'], alpha=0.07, color=p['color'])

ax1.axhline(y=len(all_sites), color='gray', linestyle=':', alpha=0.5)
ax1.text(24.1, len(all_sites), f'{len(all_sites)} N1 cells', va='center', fontsize=8, color='gray')
ax1.set_ylabel('Cells Sleeping', fontsize=10)
ax1.set_title('Closed-Loop Energy Saving — Sleeping Cells Over Time', fontsize=12, fontweight='bold')
ax1.set_ylim(bottom=0)
ax1.yaxis.set_major_locator(mticker.MaxNLocator(integer=True))
ax1.grid(axis='both', alpha=0.2, linestyle='--')

# ── Panel 2: Avg throughput over time + QoS threshold ──
ax2.plot(time_hours, summary_df['sim2_avg_thp'], color='#2E7D32', linewidth=2, label='Sim #2 Avg Throughput')
if 'sim1_avg_thp' in summary_df.columns:
    ax2.plot(time_hours, summary_df['sim1_avg_thp'], color='#FF9800', linewidth=1.5, alpha=0.6, label='Sim #1 Avg Throughput')
ax2.axhline(y=QOS_THRESHOLD, color='red', linestyle='--', linewidth=1.5, alpha=0.7, label=f'QoS Threshold ({QOS_THRESHOLD} Mbps)')

for p in _PERIODS:
    ax2.axvspan(p['start'], p['end'], alpha=0.07, color=p['color'])

ax2.set_ylabel('Avg Throughput (Mbps)', fontsize=10)
ax2.set_title('Network Throughput Over Time', fontsize=12, fontweight='bold')
ax2.legend(fontsize=9, loc='upper right')
ax2.set_ylim(bottom=0)
ax2.grid(axis='both', alpha=0.2, linestyle='--')

# Period labels on ax2
for p in _PERIODS:
    mid = (p['start'] + p['end']) / 2
    ax2.text(mid, 1.02, p['name'], ha='center', va='bottom', fontsize=8,
             fontstyle='italic', color=p['color'], alpha=0.9,
             transform=ax2.get_xaxis_transform())

ax2.set_xlabel('Time of Day (UTC)', fontsize=10)

# X-axis formatting
ax2.set_xlim(0, 24)
ax2.set_xticks(range(0, 25, 2))
ax2.set_xticklabels([f'{h:02d}:00' for h in range(0, 25, 2)], fontsize=8)

plt.tight_layout()

OUTPUT_DIR = str(PROJECT_ROOT / 'output')
os.makedirs(OUTPUT_DIR, exist_ok=True)
plt.savefig(f'{OUTPUT_DIR}/closed_loop_results.png', dpi=150, bbox_inches='tight')
plt.show()

# Print summary stats
peak_sleeping = summary_df['sleeping_cells'].max()
avg_sleeping = summary_df['sleeping_cells'].mean()
sleeping_ratio = summary_df['sleeping_cells'].mean() / (len(all_sites) * 2)
avg_thp = summary_df['sim2_avg_thp'].mean()

print(f"\nClosed-Loop Summary:")
print(f"  Peak cells sleeping: {peak_sleeping}/{len(all_sites)} N1 cells")
print(f"  Avg cells sleeping: {avg_sleeping}/{len(all_sites)} N1 cells")
print(f"  Average throughput (Sim #2): {avg_thp:.2f} Mbps")
print(f"  Approx sleeping/total ratio: {sleeping_ratio:.2f} ({sleeping_ratio*100:.0f}%)")